# Web Scraping and price estimation for Carrefour Products and Promotions  
## Présentation du projet  
**Auteurs** :  
- KAMEDA PATRICE THOMAS  
- KABORE JULIEN  
- MAMADOU GIULIANO  

**Date** : 26-04-2025  


[*LIEN VERS GITHUB*](https://github.com/patrice20035/Analyse-de-vente-de-produits/blob/main/ESTIMATION%20PAR%20FUZZY%20MATCHING)

# Introduction

## Objectif du Projet

- Scraper les **produits** et **promotions** Carrefour.
- Extraire :
  - Prix actuel et ancien prix.
  - Promotions et catégories.
- Estimer le prix d'un **produit donné** en temps réel.
- Exporter les données vers **CSV** et **Excel** avec analyses.

# Étapes du Programme
## 1. Importation des Bibliothèques

from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import os
import time
import re
from rapidfuzz import fuzz
import matplotlib.pyplot as plt

## Technologies utilisées

- **webdriver** : Contrôle du navigateur Chrome
- **Selenium** : Scraping dynamique.
- **Pandas** : Manipulation de données.
- **RapidFuzz** : Similarité entre noms de produits.
- **Matplotlib** : Graphiques.
- **Jupyter Notebook** : Développement interactif.
- **os** : Gestion des fichiers
- **import time** : Délais d'attente pour le chargement des pages
- **import re**   Expressions régulières pour nettoyer les prix

## 2. Définition des catégories Carrefour

Une liste fixe de 29 catégories est créée pour permettre à l'utilisateur de sélectionner facilement ce qu’il souhaite scraper.

```python
CATEGORIES = [
    'bio-et-ecologie', 'fruits-et-legumes', 'viandes-et-poissons', 'pains-et-patisseries',
    'cremerie-et-produits-laitiers', 'charcuterie-et-traiteur', 'surgeles', 'epicerie-salee',
    'epicerie-sucree', 'beaute-et-sante', 'boissons', 'nutrition-et-vegetale', 'hygiene-et-beaute',
    'entretien-et-nettoyage', 'animalerie', 'bebe', 'jardin', 'entretien-de-la-maison',
    'maison-et-decoration', 'cuisine', 'gros-electromenager', 'bricolage',
    'velo-trotinettes-et-loisirs', 'smartphones-et-objets-connectes', 'image-et-son',
    'informatique-et-bureau', 'jeux-videos', 'jeux-et-jouets', 'mode-et-bagagerie'
]
```

## 3. Création du dossier de sortie

```python
OUTPUT_FOLDER = r"icloud drive/Output"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
```

## 4. Interactions avec l'utilisateur
---
### Etape 1. Choix du mode utilisateur

À l'exécution, l'utilisateur choisit un des modes suivants :
    products : Scraping de produits normaux
    promotions : Scraping de promotions
    estimate : Estimation de prix par correspondance floue
Saisie contrôlée avec validation.

    
---

    
### Étape 2 : Définition des catégories Carrefour

```python
CATEGORIES = ['bio-et-ecologie', 'fruits-et-legumes', ...]
```

- Liste **prédéfinie** de catégories de produits Carrefour.
- Utilisée pour guider le scraping vers les bonnes pages.


---



##  Étape 3 : Interaction avec l'utilisateur

```python
mode = input("Souhaitez-vous scraper des promotions, des produits, ou estimer un prix ?")
```

- Trois **modes** possibles :
  1. **Promotions**.
  2. **Produits par catégorie**.
  3. **Estimation de prix**.

```python
mode = input("Souhaitez-vous scraper des promotions, des produits, ou estimer un prix ? [promotions/products/estimate] : ").strip().lower()
```

---

    
##  Étape 4 : Scraping (cas des promotions)

```python
def scrap_single_page_promotions():
    driver = webdriver.Chrome()  # Lance le navigateur Chrome
    page = 0
    while True:
        driver.get(f"https://www.carrefour.fr/promotions?page={page}")
```

- **Ouvre Chrome** et accède à la page des promotions.
- Passe en revue **chaque page** de promotions.


---


###  Fonction de scraping

**Utilisation de Selenium pour ouvrir plusieurs pages successivement**
    Récupération :
    **du nom du produit**
    **du prix proprement nettoyé**
**Sauvegarde temporaire dans un DataFrame**

```python
def scrap_pages(url_base):
    driver = webdriver.Chrome()
    all_data = []
    for page in range(5):
        driver.get(f"{url_base}&page={page}")
        time.sleep(5)
        products = driver.find_elements(By.CLASS_NAME, "product-list-grid__item")
        print(f"Scraping page {page + 1} : {len(products)} produits")

        for p in products:
            try:
                name = p.find_element(By.CSS_SELECTOR, "a.product-card-title").text
            except:
                name = None

            try:
                price_block = p.find_element(By.CSS_SELECTOR, "div.product-price__amount--main")
                price_text = price_block.text
                clean_price = re.sub(r'[^\d,\.]', '', price_text)
                price = float(clean_price.replace(',', '.'))
            except:
                price = None

            all_data.append({
                "product_name": name,
                "price": price,
                "page": page + 1
            })

    driver.quit()
    return pd.DataFrame(all_data)
```

---


##  Scraping des informations sur les produits

```python
name = p.find_element(By.CSS_SELECTOR, "a.product-card-title").text
price = float(price_text.replace(',', '.'))
old_price = float(old_price_text.replace(',', '.'))
```

- **Nom du produit**.
- **Prix actuel** et **ancien prix**.
- Nettoyage des prix avec **expressions régulières**.

---

   
### Ajout d'une description de promotion et catégorie

```python
promo_description = p.find_element(By.CSS_SELECTOR, "span.promotion-label-refonte__label").text
category = p.find_element(By.CSS_SELECTOR, "span.product-card-category").text
```

- **Description de la promotion**.
- **Catégorie du produit** (si disponible).


---

                           
## Etape 5 :  Filtrage des produits intéressants

```python
if (price and old_price and price != old_price) or promo_description:
    # Ajouter à la liste des données
```

- **Conserve** uniquement les produits qui :
  - Ont un **prix réduit**.
  - Ou une **promotion active**.


---


##  Étape 6 : Estimation de prix d’un produit

```python
def estimate_price():
    produit_cible = input("Nom du produit à estimer : ")
```

- **L'utilisateur saisit** un produit.
- Scraping de la **catégorie choisie**.


---


###  Recherche de produits similaires

```python
df['similarity'] = df['product_name'].apply(lambda x: produit_cible in x.lower())
similaires = df[df['similarity']]
```
- **Recherche par inclusion** (pas de fuzzy matching ici).
- Trouve les **produits similaires** dans la base Carrefour.     *Vérifie si le nom du produit cible est inclus dans les noms du DataFrame*

 #### Matching des produits similaires

**Fuzzy matching avec le catalogue : Compare le nom du produit en promotion avec le catalogue produit. Utilise RapidFuzz pour calculer la similarité (score de 0 à 100).**
        **Si similarité > 70%, associe la catégorie.**
            
```python       
            best_match = product_catalog.copy()
            best_match['similarity'] = best_match['product_name'].apply(lambda x: fuzz.partial_ratio(name.lower(), x.lower()) if pd.notnull(x) else 0)
            best_match = best_match.sort_values(by='similarity', ascending=False).head(1)
            category = best_match['category'].values[0] if not best_match.empty and best_match['similarity'].values[0] > 70 else None
    
    driver.quit()
    return pd.DataFrame(all_data)
```

---



#### Analyse des produits similaires

```python
prix_moyen = similaires['price'].mean()
prix_min = similaires['price'].min()
prix_max = similaires['price'].max()
```

- Affiche :
  - **Prix moyen**.
  - **Prix minimum**.
  - **Prix maximum**.



---

##  Étape 7 : Exportation des résultats

```python
df.to_csv(csv_path)
with pd.ExcelWriter(excel_path) as writer:
    df.to_excel(writer, sheet_name='Données')
```

- **CSV** pour les données brutes.
- **Excel** avec :
  - **Données brutes**.
  - **Analyses** (moyenne des réductions, nombre de promotions par catégorie).

---
##  Conclusion sur les  Fonctionnalités


1. **Scraping des promotions** (multi-pages).
2. **Scraping des produits** par catégorie.
3. **Estimation de prix** basée sur le nom d'un produit.
4. **Analyses automatiques** :
   - Produits ayant les plus grandes réductions
   -Prix min et max des produits entrés
5. **Exportation** :
   - CSV.
   - Excel (analyses incluses).


#  Perspectives d'Amélioration

**-Optimiser le matching par d'autres méthodes (embedding NLP)**.

**-Intégrer une interface utilisateur graphique (GUI).**
